In [4]:
#Import required libraries
import os
import subprocess
import sys
import torch
import cv2

In [ ]:
# Change the working directory
os.chdir('/content/drive/MyDrive/Realtime Number Plate Blurring')

if not os.path.exists('yolov9'):
  # Clone the repository if the directory doesn't exist
  !git clone https://github.com/WongKinYiu/yolov9.git
else:
  print(f"Yolov9 already exists. Skipping clone.")

Cloning into 'yolov9'...
remote: Enumerating objects: 781, done.
remote: Total 781 (delta 0), reused 0 (delta 0), pack-reused 781 (from 1)
Receiving objects: 100% (781/781), 3.27 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (330/330), done.


In [6]:
# Set the working directory
os.chdir('/content/drive/MyDrive/Realtime Number Plate Blurring/yolov9')
os.environ['WANDB_MODE'] = 'disabled'

# Install required libraries
!pip install -r requirements.txt

In [ ]:
#Load data and configurations for our dataset
def load_data_and_config():
  !cp -r ../train ./data/
  print("Loaded Train Data")
  !cp -r ../test ./data/
  print("Loaded Test Data")
  !cp -r ../valid ./data/
  print("Loaded Validation Data")
  !cp ../data.yaml ./data/
  print("Loaded Data Configuration")
  !cp ../yolov9-e.pt .
  print("Loaded pre-trained weights")
  !cp ../yolov9-e_custom.yaml ./models/detect/
  print("Loaded model configuration")
  !cp ../detect.py .
  print("Replaced detection code with bug fixes")

load_data_and_config()

Loaded Train Data
Loaded Test Data
Loaded Validation Data
Loaded Data Configuration
Loaded pre-trained weights
Loaded model configuration
Replaced detection code with bug fixes


In [ ]:
#Train the model with our dataset
def train_model(no_of_epochs):
    #Command to train the model
    command = [
        "python", "train_dual.py",
        "--workers", "8",
        "--device", "0",
        "--batch", "8",
        "--epochs", no_of_epochs,
        "--img", "640",
        "--data", "data/data.yaml",
        "--hyp", "data/hyps/hyp.scratch-high.yaml",
        "--cfg", "models/detect/yolov9-e_custom.yaml",
        "--weights", "yolov9-e.pt"
    ]

    # Run the command and stream logs
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    # Stream output line by line
    for line in process.stdout:
        sys.stdout.write(line)
        sys.stdout.flush()

    # Wait for the process to complete
    process.wait()

    # Check if the command was successful
    if process.returncode == 0:
        print(f"Training for {no_of_epochs} epochs completed successfully!")
    else:
        print(f"Training for {no_of_epochs} epochs failed with return code: {process.returncode}")
        return

    !mv ./runs/train ../exp_{no_of_epochs}
    !rm -r runs
    !rm ./data/train/labels.cache
    !rm ./data/valid/labels.cache

In [ ]:
# Train model for 50 epochs
train_model("50")

2024-12-13 18:34:49.540382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 18:34:49.562674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 18:34:49.570429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 18:34:49.588942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 18:34:50.839129: W tensorflow/comp

In [ ]:
# Train model for 100 epochs
train_model("100")

Streaming output truncated to the last 5000 lines.
      26/99      14.6G      1.433     0.8547      1.332          8        640:  78%|███████▊  | 21/27 00:26
      26/99      14.6G      1.433     0.8547      1.332          8        640:  81%|████████▏ | 22/27 00:26
      26/99      14.6G       1.44     0.8613      1.327         21        640:  81%|████████▏ | 22/27 00:27
      26/99      14.6G       1.44     0.8613      1.327         21        640:  85%|████████▌ | 23/27 00:27
      26/99      14.6G      1.447     0.8624      1.341         11        640:  85%|████████▌ | 23/27 00:28
      26/99      14.6G      1.447     0.8624      1.341         11        640:  89%|████████▉ | 24/27 00:28
      26/99      14.6G      1.445     0.8595       1.34         20        640:  89%|████████▉ | 24/27 00:29
      26/99      14.6G      1.445     0.8595       1.34         20        640:  93%|█████████▎| 25/27 00:29
      26/99      14.6G      1.448     0.8585      1.342         11        640:  93%|█

In [ ]:
# Train model for 150 epochs
train_model("150")

Streaming output truncated to the last 5000 lines.
     76/149      14.6G      1.173     0.6711       1.26         15        640:  78%|███████▊  | 21/27 00:24
     76/149      14.6G      1.173     0.6711       1.26         15        640:  81%|████████▏ | 22/27 00:24
     76/149      14.6G      1.167      0.664      1.259         16        640:  81%|████████▏ | 22/27 00:25
     76/149      14.6G      1.167      0.664      1.259         16        640:  85%|████████▌ | 23/27 00:25
     76/149      14.6G      1.168     0.6586      1.256         19        640:  85%|████████▌ | 23/27 00:27
     76/149      14.6G      1.168     0.6586      1.256         19        640:  89%|████████▉ | 24/27 00:27
     76/149      14.6G      1.162     0.6526      1.253         15        640:  89%|████████▉ | 24/27 00:28
     76/149      14.6G      1.162     0.6526      1.253         15        640:  93%|█████████▎| 25/27 00:28
     76/149      14.6G      1.166     0.6517      1.256         11        640:  93%|█

In [ ]:
# Train model for 200 epochs
train_model("200")

Streaming output truncated to the last 5000 lines.
    126/199      14.6G      1.098     0.5866      1.268         13        640:  78%|███████▊  | 21/27 00:24
    126/199      14.6G      1.098     0.5866      1.268         13        640:  81%|████████▏ | 22/27 00:24
    126/199      14.6G      1.112     0.5956      1.266          7        640:  81%|████████▏ | 22/27 00:26
    126/199      14.6G      1.112     0.5956      1.266          7        640:  85%|████████▌ | 23/27 00:26
    126/199      14.6G      1.121     0.6042       1.27         25        640:  85%|████████▌ | 23/27 00:27
    126/199      14.6G      1.121     0.6042       1.27         25        640:  89%|████████▉ | 24/27 00:27
    126/199      14.6G      1.124     0.6001      1.267         20        640:  89%|████████▉ | 24/27 00:28
    126/199      14.6G      1.124     0.6001      1.267         20        640:  93%|█████████▎| 25/27 00:28
    126/199      14.6G      1.118     0.5957      1.267         13        640:  93%|█

In [ ]:
# Train model for 250 epochs
train_model("250")

Streaming output truncated to the last 5000 lines.
    176/249      14.6G     0.9268     0.4858      1.171         11        640:  78%|███████▊  | 21/27 00:25
    176/249      14.6G     0.9268     0.4858      1.171         11        640:  81%|████████▏ | 22/27 00:25
    176/249      14.6G      0.924     0.4883      1.172         12        640:  81%|████████▏ | 22/27 00:26
    176/249      14.6G      0.924     0.4883      1.172         12        640:  85%|████████▌ | 23/27 00:26
    176/249      14.6G     0.9231     0.4888      1.172         16        640:  85%|████████▌ | 23/27 00:27
    176/249      14.6G     0.9231     0.4888      1.172         16        640:  89%|████████▉ | 24/27 00:27
    176/249      14.6G     0.9217     0.4854      1.173         14        640:  89%|████████▉ | 24/27 00:28
    176/249      14.6G     0.9217     0.4854      1.173         14        640:  93%|█████████▎| 25/27 00:28
    176/249      14.6G     0.9317     0.4878      1.182         19        640:  93%|█

In [ ]:
#Test the model
def test_model(image, weights, conf_threshold):
    #Command to test the model
    command = [
        "python", "detect.py",
        "--device", "0",
        "--weights", weights,
        "--conf", conf_threshold,
        "--img", "640",
        "--source", image
    ]

    # Run the command and stream logs
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    # Stream output line by line
    for line in process.stdout:
        sys.stdout.write(line)
        sys.stdout.flush()

    # Wait for the process to complete
    process.wait()

    # Check if the command was successful
    if process.returncode == 0:
        print(f"Test Successful!")
    else:
        print(f"Test failed with return code: {process.returncode}")

test_model("./data/test/images/23988849_jpg.rf.330fc0bdb4374b6c931bdbc375d8e444.jpg", "./exp_250/exp/weights/best.pt", "0.5")